In [2]:
# Improved Time-Series Classification Pipeline
# - Incorporates bidirectional LSTM, attention mechanism, and hyperparameter tuning
# - Adds learning rate scheduling and model checkpointing
# - Uses keras Tuner for search

import os
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load preprocessed data
data = pd.read_csv('transformed_data.csv', parse_dates=['date'] if 'date' in pd.read_csv('transformed_data.csv', nrows=1).columns else None)

# Split features and target
y = data['LogReturn']
X = data.drop(['LogReturn', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create sequences
def make_sequences(X, y, window):
    X_seq, y_seq = [], []
    for i in range(window, len(X)):
        X_seq.append(X[i-window:i])
        y_seq.append(y.iloc[i])
    return np.array(X_seq), np.array(y_seq)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Time-based split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer definition
class Attention(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
    def call(self, inputs):
        weights = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
        context = tf.reduce_sum(inputs * tf.expand_dims(weights, -1), axis=1)
        return context

# Model builder for tuning
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('units1', 32, 128, step=32), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('units2', 32, 128, step=32), return_sequences=True)))
    model.add(Attention())
    model.add(layers.Dense(hp.Int('dense', 64, 256, step=64), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(layers.Dense(1))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='mse',
        metrics=[keras.metrics.RootMeanSquaredError(), 'mae']
    )
    return model

# Hyperparameter search
import kerastuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective='val_root_mean_squared_error',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='ts_forecast'
)

tuner.search(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
               keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)]
)

# Retrieve best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.6f}, MAE: {mae:.6f}, R2: {r2:.4f}")

# Save
best_model.save('best_bidirectional_attention_model.h5')
print("Saved improved model")

Trial 10 Complete [00h 00m 16s]
val_root_mean_squared_error: 1.3376317024230957

Best val_root_mean_squared_error So Far: 1.1905879974365234
Total elapsed time: 00h 03m 40s


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 36 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


Test RMSE: 1.059721, MAE: 0.831852, R2: -0.0139
Saved improved model


In [3]:
# Improved Time-Series Classification Pipeline (Binary)
# - Classify price movement: up/down
# - Bidirectional LSTM + GRU + Attention
# - Hyperparameter tuning with Keras Tuner

import os
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('transformed_data.csv', parse_dates=['date'] if 'date' in pd.read_csv('transformed_data.csv', nrows=1).columns else None)

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=best_hp.get('tuner/epochs', 50),
    batch_size=best_hp.get('tuner/batch_size', 64),
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Save model
best_model.save('best_classification_model.keras')
print("Saved classification model")

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


TypeError: HyperParameters.get() takes 2 positional arguments but 3 were given